# Load Clean Dataset

In [33]:
import pandas as pd
df = pd.read_csv(r"C:\Users\bezis\Downloads\ethopian_bank_reviews.csv")
df.head()


,review,rating,date,bank,source
0,CBE ይለያል።,5,2025-11-29,CBE,Google Play
1,it's special for me,5,2025-11-29,CBE,Google Play
2,Make it user friendly.,2,2025-11-28,CBE,Google Play
3,maaliif daddafee install gaafata,3,2025-11-28,CBE,Google Play
4,good app,5,2025-11-28,CBE,Google Play


# STEP 2 — Sentiment Analysis (TextBlob)
- 2.1 Install Libraries
- 2.2 Analyze Sentiment
 Use TextBlob to compute polarity for each review.

Assign label:

POSITIVE if polarity > 0.05

NEGATIVE if polarity < -0.05

NEUTRAL otherwise
- 2.3 Add to DataFrame

The final no of the Assign labels
 sentiment_label
- POSITIVE    250
- NEUTRAL     190
- NEGATIVE     59
Name: count, dtype: int64

In [34]:
from textblob import TextBlob
import pandas as pd

df = pd.read_csv(r"C:\Users\bezis\Downloads\ethopian_bank_reviews.csv")

def analyze_sentiment(text):
    blob = TextBlob(str(text))
    score = blob.sentiment.polarity  # -1 to 1
    if score > 0.05:
        label = "POSITIVE"
    elif score < -0.05:
        label = "NEGATIVE"
    else:
        label = "NEUTRAL"
    return pd.Series([label, score])

df[["sentiment_label", "sentiment_score"]] = df["review"].apply(analyze_sentiment)


print(df["sentiment_label"].value_counts())
df.head()


sentiment_label
POSITIVE    250
NEUTRAL     190
NEGATIVE     59
Name: count, dtype: int64


,review,rating,date,bank,source,sentiment_label,sentiment_score
0,CBE ይለያል።,5,2025-11-29,CBE,Google Play,NEUTRAL,0.000000
1,it's special for me,5,2025-11-29,CBE,Google Play,POSITIVE,0.357143
2,Make it user friendly.,2,2025-11-28,CBE,Google Play,POSITIVE,0.375000
3,maaliif daddafee install gaafata,3,2025-11-28,CBE,Google Play,NEUTRAL,0.000000
4,good app,5,2025-11-28,CBE,Google Play,POSITIVE,0.700000


# STEP 3 — Thematic Analysis / Keyword Extraction
- 3.1 Install and Import Libraries
- 3.2 Preprocess Reviews and Extract Keywords
- 3.3 Keyword Extraction using TF-IDF
- 3.4 Group Keywords into Themes (Manual / Rule-Based)
- 3.5 Preview and Save Results

In [35]:
# Install if not already
# !pip install spacy sklearn

import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

# Load English tokenizer, POS tagger, etc.
nlp = spacy.load("en_core_web_sm")


In [36]:
import spacy
nlp = spacy.load("en_core_web_sm")

def extract_nouns(text):
    doc = nlp(str(text))
    nouns = [token.lemma_.lower() for token in doc if token.pos_ in ["NOUN", "PROPN"]]
    return nouns

df['nouns'] = df['review'].apply(extract_nouns)


In [37]:
# Map keywords to themes
theme_mapping = {
    "User Interface & Experience": ["ui", "design", "experience", "user", "application", "app"],
    "Account Access Issues": ["access", "account", "password", "pin"],
    "Transaction Performance": ["transfer", "transaction", "money", "bill", "balance"],
    "Customer Support": ["service", "developer", "help", "thank", "super"],
    "Feature Requests / Updates": ["update", "feature", "option", "system", "mode"]
}

# Function to assign themes to each review
def assign_theme(nouns):
    assigned = []
    for theme, words in theme_mapping.items():
        if any(word in nouns for word in words):
            assigned.append(theme)
    if not assigned:
        assigned.append("Other")
    return ", ".join(assigned)

df['identified_theme'] = df['nouns'].apply(assign_theme)


In [39]:
# Keep only relevant columns for output
output_df = df[['review', 'rating', 'bank', 'sentiment_label', 'sentiment_score', 'identified_theme']]

# Preview
output_df.head(10)

# Save to CSV
output_df.to_csv("bank_reviews_with_sentiment_and_themes.csv", index=False)
df.head()


,review,rating,date,bank,source,sentiment_label,sentiment_score,nouns,identified_theme
0,CBE ይለያል።,5,2025-11-29,CBE,Google Play,NEUTRAL,0.000000,"[cbe, ይለያል።]",Other
1,it's special for me,5,2025-11-29,CBE,Google Play,POSITIVE,0.357143,[],Other
2,Make it user friendly.,2,2025-11-28,CBE,Google Play,POSITIVE,0.375000,[],Other
3,maaliif daddafee install gaafata,3,2025-11-28,CBE,Google Play,NEUTRAL,0.000000,"[maaliif, daddafee, install, gaafata]",Other
4,good app,5,2025-11-28,CBE,Google Play,POSITIVE,0.700000,[app],User Interface & Experience


# STEP 4 — Aggregate Sentiment by Bank + Rating
🔹 Sentiment by Bank
🔹 Sentiment by Rating
🔹 BOA — 0.1697

Lowest average sentiment

Users are more neutral or slightly positive

Indicates more complaints or mixed experiences

🔹 CBE — 0.2275

Middle score

Users lean positive, but not strongly

Feedback is a mix of praise and pain points

🔹 Dashen — 0.2439

Highest average sentiment

Users feel more positive compared to others

Still not extremely positive, but relatively better overall satisfaction

Conclusion

Lower ratings → negative sentiment

Higher ratings → positive sentiment

This validates your sentiment analysis and ensures the model is reliable

In [40]:
sentiment_by_bank = df.groupby("bank")["sentiment_score"].mean()
sentiment_by_bank


bank
BOA       0.169753
CBE       0.227502
Dashen    0.243984
Name: sentiment_score, dtype: float64

In [41]:
df.groupby("rating")["sentiment_score"].mean()


rating
1   -0.132977
2   -0.023505
3    0.155253
4    0.195720
5    0.393425
Name: sentiment_score, dtype: float64

# Step 5  Top Keywords per Bank
Identify important words that appear in user reviews for each bank to understand main topics, pain points, or praised features.

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a column with all nouns in each review as a string
df['noun_text'] = df['nouns'].apply(lambda x: ' '.join(x))

top_keywords_per_bank = {}

# Loop through each bank
for bank in df['bank'].unique():
    texts = df[df['bank'] == bank]['noun_text'].tolist()
    tfidf = TfidfVectorizer(max_features=20)  # top 20 keywords per bank
    tfidf_matrix = tfidf.fit_transform(texts)
    top_keywords_per_bank[bank] = tfidf.get_feature_names_out()

# Display results
top_keywords_per_bank


{'CBE': array(['app', 'application', 'bank', 'banking', 'branch', 'card', 'cbe',
        'customer', 'money', 'option', 'service', 'statement', 'time',
        'transaction', 'update', 'user', 'በጣም', 'ብር', 'ነው', 'ግን'],
       dtype=object),
 'BOA': array(['app', 'application', 'bank', 'banking', 'boa', 'branch', 'bug',
        'developer', 'ethiopia', 'experience', 'issue', 'mobile', 'money',
        'number', 'option', 'password', 'phone', 'problem', 'time',
        'update'], dtype=object),
 'Dashen': array(['account', 'app', 'bank', 'banking', 'dashen', 'design',
        'ethiopia', 'experience', 'feature', 'issue', 'money', 'service',
        'super', 'time', 'transaction', 'transfer', 'ui', 'update', 'user',
        'work'], dtype=object)}

# Step 6 Theme Assignment per Bank


In [45]:
themes_per_bank = df.groupby(["bank", "identified_theme"]).size().unstack(fill_value=0)
themes_per_bank


identified_theme,Account & Transactions,Customer Support,Other,Technical Issues,User Interface & Experience
bank,,,,,
BOA,18,0,93,3,53
CBE,10,5,100,0,47
Dashen,5,2,71,0,92


In [48]:
df.to_csv("task2_sentiment_themes.csv", index=False)
df.head()


,review,rating,date,bank,source,sentiment_label,sentiment_score,nouns,identified_theme,noun_text
0,CBE ይለያል።,5,2025-11-29,CBE,Google Play,NEUTRAL,0.000000,"[cbe, ይለያል።]",Other,cbe ይለያል።
1,it's special for me,5,2025-11-29,CBE,Google Play,POSITIVE,0.357143,[],Other,
2,Make it user friendly.,2,2025-11-28,CBE,Google Play,POSITIVE,0.375000,[],Other,
3,maaliif daddafee install gaafata,3,2025-11-28,CBE,Google Play,NEUTRAL,0.000000,"[maaliif, daddafee, install, gaafata]",Other,maaliif daddafee install gaafata
4,good app,5,2025-11-28,CBE,Google Play,POSITIVE,0.700000,[app],User Interface & Experience,app


## Step 7 — Conclusions from Thematic Analysis

Based on the theme assignment across banks, the following observations can be made:

### 1. Most Discussed Themes per Bank
- **BOA:** High mentions of *Other* (93) and *User Interface & Experience* (53) — users focus on app experience, with miscellaneous issues also present.
- **CBE:** High mentions of *Other* (100) and *User Interface & Experience* (47) — app usability is important; miscellaneous issues exist.
- **Dashen:** Predominantly *User Interface & Experience* (92) — strong focus on app usability and design.

### 2. Minor Themes
- **Account & Transactions:** Some users report transaction issues, notably BOA (18) and CBE (10).
- **Customer Support:** Very few mentions, indicating either minor issues or underreporting.
- **Technical Issues:** Only BOA (3) mentions technical problems, suggesting generally stable apps.

### 3. Actionable Insights
1. **Enhance UI/UX:** Dashen and CBE should continue improving app usability, as it drives satisfaction.
2. **Investigate “Other” issues:** Large counts in BOA and CBE indicate areas needing deeper review.
3. **Monitor Transactions & Support:** Minor but important issues should be addressed to ensure reliability.
4. **Technical Stability:** BOA should prioritize bug fixes to prevent negative user experiences.

> **Summary:** Overall, **User Interface & Experience** is the dominant theme influencing satisfaction, while miscellaneous issues may hide other concerns. Banks should focus on maintaining excellent UI/UX while resolving less frequent but important user issues.
